In [1]:
from sklearn.externals import joblib
import numpy as np
from PIL import Image

In [2]:
X = joblib.load("/home/aanax/Desktop/kino/0.pkl")
X_test = joblib.load("/home/aanax/Desktop/kino/1.pkl")
y = joblib.load("/home/aanax/Desktop/kino/2.pkl")
y_test = joblib.load("/home/aanax/Desktop/kino/3.pkl")

/home/aanax/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: DeprecationWarning: The file '/home/aanax/Desktop/kino/0.pkl' has been generated with a joblib version less than 0.10. Please regenerate this pickle file.
  if __name__ == '__main__':
/home/aanax/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: The file '/home/aanax/Desktop/kino/1.pkl' has been generated with a joblib version less than 0.10. Please regenerate this pickle file.
  from ipykernel import kernelapp as app
/home/aanax/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: The file '/home/aanax/Desktop/kino/2.pkl' has been generated with a joblib version less than 0.10. Please regenerate this pickle file.
  app.launch_new_instance()
/home/aanax/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: DeprecationWarning: The file '/home/aanax/Desktop/kino/3.pkl' has been generated with a joblib version less than 0.10. Please regenerate 

In [4]:
def showpic(digit,multiplier=1,size=200):
    tstimage1= Image.fromarray(digit.reshape((30,20)) * multiplier)
    tstimage1 = tstimage1.convert('L')
    tstimage1 = tstimage1.resize((size,size))
    tstimage1.show()

In [7]:
def crop_showpic(digit,multiplier=1,size=200):
    tstimage1= Image.fromarray(digit.reshape((20,20)) * multiplier)
    tstimage1 = tstimage1.convert('L')
    tstimage1 = tstimage1.resize((size,size))
    tstimage1.show()

In [9]:
showpic(X[5])

In [59]:
means = []
for pic in X:
    means.append(np.array(pic).mean())

In [82]:
# test dummy solution (nearly 0.6 acc on test)
test_p = []
for me in means:
    if me > 159:
        test_p.append(2)
    else:
        test_p.append(1)

In [14]:
def eval(test_p,y,ver,wrong):
    t = 0 #true
    fp=0 # false positive 
    fn=0 # false negative
    tp=0 # true positive
    tn=0 # true negative
    fals = max(y)
    tru =min(y)
    kkk = -1 # counter
    for i,j in zip(test_p,y):
        kkk+=1
        if i==j:
            t+=1
            if i > tru:
                tn +=1
            if i < fals:
                tp+=1
        if i>j:
            fn+=1
            wrong.append(kkk)
        if i<j:
            fp+=1
            wrong.append(kkk)
    if ver!=0:
        print(" fn ",fn," fp ",fp," tn ",tn," tp ",tp," acc ",float(t)/len(y))

    return (float(t)/len(y))

In [307]:
grads = []
for pic in X:
    grads.append(np.gradient(pic,2))
    
grads_test = []
for pic in X_test:
    grads_test.append(np.gradient(pic,2))

In [15]:
def threshme(test_p,X,thresh):
    for me in X:
        if me < thresh:
            test_p.append(2)
        else:
            test_p.append(1)

In [67]:
import xgboost as xgb


/home/aanax/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [377]:

boosters = []
n_boosters = 1

for i in range(0, n_boosters):
    #Xtr, Xval, Ytr, Yval = train_test_split(X_train, Y_train, test_size=0.2)#,random_state=4455)
    dmat_train = xgb.DMatrix(X, np.array(y)-1)
    dmat_test = xgb.DMatrix(X_test, np.array(y_test)-1)
    
    params = {"objective": "binary:logistic",
              
          "eta": 0.001,
          "max_depth": 5,
          "eval_metric": "auc", }
    bst1 = xgb.train(dtrain=dmat_train, params=params, num_boost_round=100000, early_stopping_rounds=50, verbose_eval=100, evals=[(dmat_test, "val_0")])
    
    boosters.append(bst)

[0]	val_0-auc:0.838424
Will train until val_0-auc hasn't improved in 50 rounds.
Stopping. Best iteration:
[4]	val_0-auc:0.87122



In [568]:
grads = []
grads_test = []
gradssums = []
for pic in X:
    gradssums.append(list(np.gradient(pic,2)) + [pic.sum()])
gradssums_test = []
for pic in X_test:
    gradssums_test.append(list(np.gradient(pic,2)) + [pic.sum()])
        

In [573]:
gradssums = np.array(gradssums)
gradssums_test = np.array(gradssums_test)

In [30]:
allx = np.vstack((X,X_test)) 

In [100]:
y_tst = y_test
ones = list(y_tst).count(1)
twos = list(y_tst).count(2)
nones = 0
ntwos = 0
todel = []
for i in range(len(y_tst)):
    #if y_test[i] == 1:
    #    nones += 1
    if y_test[i] == 2:
        ntwos += 1
        if ntwos > ones:
            todel.append(i)
y_tst = np.delete(y_tst,todel)
X_tst = np.delete(X_test,todel,0)        
    

In [101]:
ones = list(y_tst).count(1)
twos = list(y_tst).count(2)

In [141]:
# import the necessary packages
from skimage import feature
import numpy as np
 
class LocalBinaryPatterns:
	def __init__(self, numPoints, radius):
		# store the number of points and radius
		self.numPoints = numPoints
		self.radius = radius
 
	def describe(self, image, eps=1e-7):
		# compute the Local Binary Pattern representation
		# of the image, and then use the LBP representation
		# to build the histogram of patterns
		lbp = feature.local_binary_pattern(image, self.numPoints,
			self.radius, method="uniform")
		(hist, _) = np.histogram(lbp.ravel(),
			bins=np.arange(0, self.numPoints + 3),
			range=(0, self.numPoints + 2))
 
		# normalize the histogram
		hist = hist.astype("float")
		hist /= (hist.sum() + eps)
 
		# return the histogram of Local Binary Patterns
		return hist

In [142]:
desc = LocalBinaryPatterns(24, 8)

In [145]:
hist = desc.describe(X[0].reshape((30,20)))

In [155]:
grads = []
for pic in X:
    grads.append(list(np.gradient(pic, 1)) + [ndimage.fourier.fourier_ellipsoid(pic,3).sum(),ndimage.center_of_mass(pic)[0], ndimage.extrema(pic)[3][0], pic.reshape((30,20)).diagonal().sum(), pic.reshape((30,20)).diagonal(offset=1).sum() ] + list(ndimage.gaussian_laplace(pic,2.7))   )#+ list(desc.describe(pic.reshape((30,20)))))
    
grads_test = []
for pic in X_tst:
    grads_test.append(list(np.gradient(pic, 1))+ [ndimage.fourier.fourier_ellipsoid(pic,3).sum(), ndimage.center_of_mass(pic)[0], ndimage.extrema(pic)[2][0], pic.reshape((30,20)).diagonal().sum(), pic.reshape((30,20)).diagonal(offset=1).sum()] + list(ndimage.gaussian_laplace(pic,2.7)) ) # + list(desc.describe(pic.reshape((30,20)))) )

In [134]:
X[0].reshape((30,20)).diagonal(offset = 1,axis1=1,axis2=0)

array([ 78, 105, 147, 159, 162, 166, 166, 159, 148, 134, 130, 123, 121,
       122, 121, 120, 131, 134, 137, 139], dtype=uint8)

In [858]:
showpic(X[0])
showpic(ndimage.gaussian_laplace(X[0],2.5))

In [156]:
grads = np.array(grads)
grads_test = np.array(grads_test)

In [17]:
from scipy import ndimage #import filters

In [157]:
from sklearn.model_selection import train_test_split
boosters = []
n_boosters = 1

for i in range(0, n_boosters):
    #Xtr, Xval, Ytr, Yval = train_test_split(grads, np.array(y)-1, test_size=0.2)#,random_state=4455)
    dmat_train = xgb.DMatrix(grads, np.array(y)-1)
    dmat_test = xgb.DMatrix((grads_test), np.array(y_tst)-1)
        
    params = {"objective": "binary:logistic",
                
          "eta": 0.001,
          "lambda": 5,
          "alpha": 5,
          "max_depth": 10,
          "eval_metric": "auc", }
    bst = xgb.train(dtrain=dmat_train, params=params, num_boost_round=100000, early_stopping_rounds=50, verbose_eval=100, evals=[(dmat_test, "val_0")])
    
    boosters.append(bst)

[0]	val_0-auc:0.985873
Will train until val_0-auc hasn't improved in 50 rounds.
[100]	val_0-auc:0.996676
Stopping. Best iteration:
[113]	val_0-auc:0.997895



In [158]:
preds = np.array([bst.predict(xgb.DMatrix(grads_test)) for bst in boosters])
preds = np.mean(preds, axis=0)

In [159]:
p=[]
threshme(p,preds,0.5)

In [160]:
# evaluate xgboost and remember misclassified examples to look at them
wrong=[]
eval(np.array(p)-1,(np.array(y_tst)-2)*-1,1,wrong)

 fn  0  fp  6  tn  89  tp  95  acc  0.968421052631579


0.968421052631579

In [164]:
for i in wrong:
    showpic(X[i])

In [ ]:
#TRY ANSAMBLE WITH GRADSSUMS